In [ ]:
import pandas as pd

import io
import re
import os

import math

pd.options.display.max_columns = None
pd.options.display.max_rows = None

triples = pd.DataFrame(columns=['e1', 'rel', 'e2'])


In [ ]:
# create folders for each annotator
annotators = []
for path, dir, files in os.walk("./trafficking-culture-annotations/annotation"):
    for name in files:
        # makes sure annotators are unique and removes hidden files (ie .DS_Store) 
        if name not in annotators and name[0] != '.':
            annotators.append(name)


if not os.path.exists('anno_by_person'):
    os.mkdir('anno_by_person')

for i in annotators:
    folder_path = 'anno_by_person/' + os.path.splitext(i)[0]
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)


In [ ]:
# grab only the lines that include data

if not os.path.exists('conversion_output'):
    os.mkdir('conversion_output')


def convert(path):
    all_lns = ''
    nl = ''
    with open(path, 'r') as f:
        for line in f:
            if '\t' in line:
                nl = line.rstrip()
                all_lns += nl + '\n'
                
    return(io.StringIO(all_lns))

for folder in os.listdir('./anno_by_person/'):
    print(folder)
    if folder[0] != '.':
        folder_path = './anno_by_person/' + folder + '/'
        for file in os.listdir(folder_path):
            if not file.startswith('.'):
                data = pd.read_csv(convert(folder_path + file), sep='\t', names=['token_id', 'offset_pos', 'text', 'entity_tag', 'relation_tag', 'related_token'])

                only_entities = data[data.entity_tag != '_']
                only_entities = only_entities.fillna('no-rel')
                print(only_entities)

                def full_entity(ent):
                    ent_tag = only_entities.loc[only_entities.token_id == ent,'entity_tag']    
                    et = ent_tag.to_string(index=False)
                    
                    full_ent = ''
                    
                    if '[' in et:
                        for tag in only_entities.index:
                            if only_entities['entity_tag'][tag] == et:
                                full_ent += only_entities['text'][tag] + ' '
                        return(full_ent)
                    else:
                        ent_txt = only_entities.loc[only_entities.token_id == ent,'text']
                        ent_txt = ent_txt.to_string(index=False)
                        return(ent_txt)     

                for i in only_entities.index:
                    if only_entities['relation_tag'][i] != '_' and only_entities['relation_tag'][i] != 'no-rel':
                        e1 = only_entities['token_id'][i]
                        e1t = full_entity(e1)
                        relation = only_entities['relation_tag'][i]
                        e2 = only_entities['related_token'][i]
                        
                        print(file)
                        print(relation)
                        if '|' in relation:
                            ls_rel = relation.split('|')
                            ls_e2 = e2.split('|')
                            for j in range(len(ls_rel)):
                                tid = ls_e2[j].split('[')[0]
                                e2t = full_entity(tid)
                                
                                ls_rel[j] = ls_rel[j].replace('\\', '')
                                                        
                                triple = e2t.strip() + '|' + ls_rel[j].strip() + '|' + e1t.strip()
                                print(triple)


                                mk_ls = triple.split('|')
                                print(mk_ls)
                                triples.loc[len(triples)] = mk_ls    
                        else:       
                            tid = e2.split('[')[0]
                            e2t = full_entity(tid)
                            e2t = re.sub(r'[^\w\s]', '', e2t).strip()
                    
                            relation = relation.replace('\\', '')
                            
                            triple = e2t.strip() + '|' + relation.strip() + '|' + e1t.strip()
                            print(triple)

                            mk_ls = triple.split('|')
                            print(mk_ls)
                            triples.loc[len(triples)] = mk_ls

        export_filename = 'conversion_output/' + folder + '_triples.csv'

        triples.to_csv(export_filename, index=False, header=False)
        triples